# SSMR: generate SSM model from trajectory data

In [37]:
import numpy as np
from copy import deepcopy
from os.path import join
import pickle
from os.path import dirname, abspath, join
import sys
from os import listdir, mkdir, getcwd
from os.path import join, exists, isdir

In [38]:
%matplotlib qt
import matplotlib.pyplot as plt

In [39]:
%load_ext autoreload
%autoreload 2
import ROM.python.utils as utils
import plot_utils as plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
np.set_printoptions(linewidth=300)
path = getcwd()
root = dirname(path)
sys.path.append(root)

## Settings for SSM model

In [41]:
SETTINGS = {
    'observables': "delay-embedding", # "pos-vel", # "delay-embedding"
    'reduced_coordinates': "local", # "global" # "local"
    'control_observables': "pos-vel",
    'use_ssmlearn': "matlab", # "matlab", "py"

    'tip_node': 1354,
    'n_nodes': 1628,
    'input_dim': 4,
    
    'dt': 0.01,
    'subsample': 1,

    'rDOF': 3,
    'oDOF': 3,
    'n_delay': 6,
    'SSMDim': 6,
    'SSMOrder': 3,
    'ROMOrder': 3,
    'RDType': "flow",
    'ridge_alpha': {
        'manifold': 0., # 1.,
        'reduced_dynamics': 0., # 100.,
        'B': 0. # 1.
    },
    'custom_delay': None, # Specify a custom delay embedding for the SSM

    'data_dir': "/home/jalora/SSMR-for-control/ROM/python/hardware",
    # 'data_dir': "/home/jalora/Desktop/diamond_origin",
    'data_subdirs': False,
    'decay_dir': "decay/",
    'rest_file': "rest_qv.pkl",
    'model_save_dir': "SSMmodelsPy/",

    't_decay': [1, 4],
    't_truncate': [0.25, np.inf],

    'decay_test_set': [18, 19, 33],
    'decay_holdout_set': [11],

    'traj_test_set': [],
    'traj_holdout_set': [0, 1, 2],

    'poly_u_order': 1,
    'input_train_data_dir': "open-loop",
    'input_test_data_dir': [],
    'input_train_ratio': 0.8 # 0.8
}
SETTINGS['data_subdirs'] = sorted([dir for dir in listdir(SETTINGS['data_dir']) if isdir(join(SETTINGS['data_dir'], dir)) and
                                   'decay' in listdir(join(SETTINGS['data_dir'], dir)) and
                                   'open-loop' in listdir(join(SETTINGS['data_dir'], dir))])
print(SETTINGS['data_subdirs'])
PLOTS = "show"

[]


## Import and inspect decay data

In [42]:
# create a new directory into which the new model will be saved and get the equilibrium state
model_save_dir = path
decay_data_dir = path
q_eq = np.zeros(SETTINGS['oDOF'])
u_eq = np.zeros(4)

obsNodes = [12, 13, 14, 27, 28, 29]
outdofs = [3, 4, 5] # TODO: Figure out where this goes
idxTrajRegressC = 10

# ====== Import decay trajectories -- oData ====== #
print("====== Import decay trajectories ======")    
Data = {}
# For /home/jjalora/Desktop/Diamond
Data['oData'] = utils.import_pos_data(decay_data_dir, 
                                        q_rest=q_eq, 
                                        output_node=obsNodes, 
                                        file_type='mat', 
                                        return_velocity=False,
                                        hardware=True)
nTRAJ = len(Data['oData'])
print(nTRAJ)

====== Import decay trajectories ======
34


## Observables

EITHER compute reduced coordinates using delay embedding on the available oData OR use position and velocity of all nodes as observables

In [43]:
def assemble_observables(oData):
    return utils.delayEmbedding(oData, embed_coords=np.arange(3), up_to_delay=SETTINGS['n_delay'])

Compute observables

In [44]:
Data['yData'] = deepcopy(Data['oData'])
for i in range(nTRAJ):
    Data['yData'][i][1] = assemble_observables(Data['oData'][i][1])

### Plot observables of interest

In [46]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = [] # [14, 16, 18, 31, 35, 44] # []
plot.traj_xyz(Data,
             xyz_idx=[('oData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
             highlight_idx=highlight_traj)

Slice trajectories, i.e. remove time before they converge with SSM and cut off the end after complete decay

In [47]:
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], SETTINGS['t_truncate'], t_shift=SETTINGS['n_delay'] * SETTINGS['dt'])
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], SETTINGS['t_truncate'], t_shift=SETTINGS['n_delay'] * SETTINGS['dt'])

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [56]:
svd_data = 'yDataTrunc'
Data['etaDataTrunc'] = deepcopy(Data['oDataTrunc'])
show_modes = 9
Xsnapshots = np.hstack([DataTrunc[1] for DataTrunc in Data[svd_data]])

v, s = utils.sparse_svd(Xsnapshots, up_to_mode=max(SETTINGS['SSMDim'], show_modes))
Vde = v[:, :SETTINGS['SSMDim']]

for i in range(nTRAJ):
    Data['etaDataTrunc'][i][1] = Vde.T @ Data[svd_data][i][1]

# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(s**2, up_to_mode=show_modes)

In [57]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_xyz(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
              xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'])

### Train and test data (train/test split)

In [58]:
indTest = SETTINGS['decay_test_set']
indHoldout = SETTINGS['decay_holdout_set']
indTrain = [i for i in range(nTRAJ) if i not in [*indTest, *indHoldout]]

## SSMLearnPy

In [59]:
from ssmlearnpy import SSMLearn

Use SSMLearnPy package to find parametrization of SSM (graph style) and reduced dynamics on the SSM.

In [60]:
from scipy.linalg import orth

# Regress current reduced coordinates with new observable (used to infer the linear part of manifold, i.e., tangent space)
ssm_paramonly = SSMLearn(
t=[Data['oDataTrunc'][i][0] for i in indTrain], 
x=[Data['oDataTrunc'][i][1] for i in indTrain], 
reduced_coordinates=[Data['etaDataTrunc'][i][1] for i in indTrain],
ssm_dim=SETTINGS["SSMDim"], 
dynamics_type=SETTINGS["RDType"]
)
ssm_paramonly.get_parametrization(poly_degree=SETTINGS["SSMOrder"], alpha=SETTINGS["ridge_alpha"]['manifold'])

# # Rotation of coordinates
# # Calculate tangent space at origin and orthogonalize
# tanspace0_not_orth = ssm_paramonly.decoder.map_info['coefficients'][:SSMDim, :SSMDim]
# tanspace0 = orth(tanspace0_not_orth)

# # Change reduced coordinates and repopulate eta
# Data['etaDataTruncNew'] = deepcopy(Data['etaDataTrunc'])
# for iTraj in range(len(Data['etaDataTruncNew'])):
#     Data['etaDataTruncNew'][iTraj][1] = np.transpose(tanspace0) @ tanspace0_not_orth @ Data['etaDataTrunc'][iTraj][1]
# Data['etaDataTrunc'] = deepcopy(Data['etaDataTruncNew'])
# Data.pop('etaDataTruncNew', None)

# # Regress the new reduced coordinates with the new observable (used to infer the nonlinear part of manifold)
# # Get the parameterization
# ssm_paramonly = SSMLearn(
# t=[Data['yDataTrunc'][i][0] for i in indTrain], 
# x=[Data['yDataTrunc'][i][1] for i in indTrain], 
# reduced_coordinates=[Data['etaDataTrunc'][i][1] for i in indTrain],
# ssm_dim=SSMDim, 
# dynamics_type=RDType
# )
# ssm_paramonly.get_parametrization(poly_degree=SSMOrder, alpha=ridge_alpha['manifold'])

# Construct the chart
ssm_chartonly = SSMLearn(
    t=[Data['etaDataTrunc'][i][0] for i in indTrain], 
    x=[Data['etaDataTrunc'][i][1] for i in indTrain], 
    reduced_coordinates=[Data['oDataTrunc'][i][1] for i in indTrain],
    ssm_dim=SETTINGS["SSMDim"],
    dynamics_type=SETTINGS["RDType"],
)
ssm_chartonly.get_parametrization(poly_degree=SETTINGS["SSMOrder"], alpha=SETTINGS["ridge_alpha"]['manifold'])

# Reassign to get new rotated reduced dynamics
ssm = ssm_paramonly

INFO   2023-09-13 08:28:28 ridge Transforming data
INFO   2023-09-13 08:28:28 ridge Skipping CV on ridge regression
INFO   2023-09-13 08:28:28 ridge Fitting regression model


INFO   2023-09-13 08:28:29 ridge Transforming data
INFO   2023-09-13 08:28:29 ridge Skipping CV on ridge regression
INFO   2023-09-13 08:28:29 ridge Fitting regression model


In [61]:
ssm.get_parametrization(poly_degree=SETTINGS["SSMOrder"], alpha=SETTINGS["ridge_alpha"]['manifold'])    
ssm.get_reduced_dynamics(poly_degree=SETTINGS["ROMOrder"], alpha=SETTINGS["ridge_alpha"]['reduced_dynamics'])

INFO   2023-09-13 08:28:35 ridge Transforming data
INFO   2023-09-13 08:28:35 ridge Skipping CV on ridge regression
INFO   2023-09-13 08:28:35 ridge Fitting regression model


INFO   2023-09-13 08:28:35 ridge Transforming data
INFO   2023-09-13 08:28:35 ridge Skipping CV on ridge regression
INFO   2023-09-13 08:28:35 ridge Fitting regression model


Save the relevant coefficents and parameters into dictionairies which resemble the outputs of the Matlab SSMLearn package.

In [62]:
IMInfo = {'parametrization': {}, 'chart': {}}
IMInfo['parametrization']['polynomialOrder'] = SETTINGS["SSMOrder"]
IMInfo['parametrization']['H'] = ssm.decoder.map_info['coefficients']
RDInfo = {'reducedDynamics': {}}
RDInfo['reducedDynamics']['polynomialOrder'] = SETTINGS["ROMOrder"]
RDInfo['reducedDynamics']['coefficients'] = ssm.reduced_dynamics.map_info['coefficients']
RDInfo['eigenvaluesLinPartFlow'] = ssm.reduced_dynamics.map_info['eigenvalues_lin_part']
RDInfo['dynamicsType'] = SETTINGS["RDType"]

In [63]:
IMInfo['chart']['polynomialOrder'] = SETTINGS["SSMOrder"]
IMInfo['chart']['H'] = ssm_chartonly.decoder.map_info['coefficients']

## SSMLearn (Matlab)

Start Matlab engine and install/run SSMLearn

In [181]:
# ssm = utils.start_matlab_ssmlearn("/home/jonas/Projects/stanford/SSMR-for-control")

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [182]:
# yDataTruncTrain_matlab = utils.numpy_to_matlab([Data['yDataTrunc'][i] for i in indTrain])
# etaDataTruncTrain_matlab = utils.numpy_to_matlab([Data['etaDataTrunc'][i] for i in indTrain])

### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [183]:
# IMInfo = ssm.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
#                         'reducedCoordinates', etaDataTruncTrain_matlab, 'l_vals', 0.)
# if observables == "pos-vel":
#     IMInfoInv = ssm.IMGeometry(etaDataTruncTrain_matlab, obsDim, SSMOrder,
#                             'reducedCoordinates', yDataTruncTrain_matlab)
#     for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
#         IMInfo['chart'][key] = IMInfoInv['parametrization'][key]

### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [184]:
# RDInfo = ssm.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default', 'l_vals', 0.)

Stop Matlab engine -- not needed anymore

In [185]:
# ssm.quit()

Convert all matlab double arrays to numpy arrays

In [186]:
# utils.matlab_info_dict_to_numpy(IMInfo)
# utils.matlab_info_dict_to_numpy(RDInfo)

Stability analysis of reduced dynamics

In [187]:
# # assert np.all(np.real(RDInfo['eigenvaluesLinPartFlow']) < 0)
# RDInfo['eigenvaluesLinPartFlow']

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [65]:
trajRec = {}
# geometry error
meanErrorGeo = {}
trajRec['geo'] = utils.lift_trajectories(IMInfo, Data['etaDataTrunc'])
normedTrajDist = utils.compute_trajectory_errors(trajRec['geo'], Data['oDataTrunc'])[0] * 100
meanErrorGeo['Train'] = np.mean(normedTrajDist[indTrain])
meanErrorGeo['Test'] = np.mean(normedTrajDist[indTest])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

Average parametrization train error: 1.3364e+00
Average parametrization test error: 1.4711e+00


In [67]:
# plot comparison of SSM-predicted vs. actual test trajectories
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(trajRec,
            xyz_idx=[('geo', 0), ('geo', 1), ('geo', 2)],
            xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
            traj_idx=indTest,
            axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [68]:
# reduced dynamics error
meanErrorDyn = {}
trajRec['rd'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
normedTrajDist = utils.compute_trajectory_errors(trajRec['rd'], Data['etaDataTrunc'])[0] * 100
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain])
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest])
print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

Average dynamics train error: 5.2354
Average dynamics test error: inf


In [69]:
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(trajRec,
            xyz_idx=[('rd', 0), ('rd', 1), ('rd', 2)],
            xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'],
            traj_idx=indTest,
            axs=axs, ls=':', color='darkblue')

### Global error

In [71]:
# global error
meanErrorGlo = {}
trajRec['glob'] = utils.lift_trajectories(IMInfo, trajRec['rd'])
normedTrajDist = utils.compute_trajectory_errors(trajRec['glob'], Data['oDataTrunc'])[0] * 100
meanErrorGlo['Train'] = np.mean(normedTrajDist[indTrain])
meanErrorGlo['Test'] = np.mean(normedTrajDist[indTest])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global train error: 4.3995
Average global test error: inf


In [72]:
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(trajRec,
            xyz_idx=[('glob', 0), ('glob', 1), ('glob', 2)],
            xyz_names=[r'$x$', r'$y$', r'$z$'],
            traj_idx=indTest,
            axs=axs, ls=':', color='darkblue')

## Control

### Setup model for control

In [81]:
Rauton = lambda x: RDInfo['reducedDynamics']['coefficients'] @ utils.phi(x, RDInfo['reducedDynamics']['polynomialOrder'])
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.phi(x, IMInfo['parametrization']['polynomialOrder'])
Wauton = lambda y: IMInfo['chart']['H'] @ utils.phi(y, IMInfo['chart']['polynomialOrder'])

### Learn control matrix $B$

Use randomly sampled inputs as the training and test data

In [76]:
outData, inputData = utils.import_traj_data(join(path, SETTINGS['input_train_data_dir']), obsNodes)

Compute observables and reduced coordinates

In [77]:
y = outData
for i in range(len(outData)):
    y[i][1] = outData[i][1]

indCntrlTest = SETTINGS['traj_test_set']
indCntrlHoldout = SETTINGS['traj_holdout_set']
indCntrlTrain = [i for i in range(np.shape(outData)[0]) if i not in [*indCntrlTest, *indCntrlHoldout]]

Train/test split

In [83]:
zData, yData, uData = [], [], []
for iTraj in indCntrlTrain:
    yData.append(y[iTraj][1])
    uData.append(inputData[iTraj][1])
    zData.append(outData[iTraj][1])

yDataFull = np.concatenate(yData, axis=1)
uDataFull = np.concatenate(uData, axis=1)
zDataFull = np.concatenate(zData, axis=1)
xFull = Wauton(yDataFull)
tFull = np.arange(0, np.shape(yDataFull)[1]*SETTINGS['dt'], SETTINGS['dt'])

Control Data train/test split

In [84]:
split_idx = int(SETTINGS['input_train_ratio'] * len(tFull))
t_train, t_test = tFull[:split_idx], tFull[split_idx:]
_, z_test = zDataFull[:, :split_idx], zDataFull[:, split_idx:]
# y_train, y_test = y[:, :split_idx], y[:, split_idx:]
u_train, u_test = uDataFull[:, :split_idx], uDataFull[:, split_idx:]
x_train, x_test = xFull[:, :split_idx], xFull[:, split_idx:]

controlData = {'t_train': t_train, 't_test': t_test, 'z_test': z_test, 'u_train': u_train, 'u_test': u_test, 
        'x_train': x_train, 'x_test': x_test, 'dxdt': np.gradient(x_train, SETTINGS['dt'], axis=1), 'dxdt_ROM': Rauton(x_train)}

Learn $B$ matrix

In [86]:
Br = utils.fitControlMatrix(Rauton, x_train, u_train, SETTINGS['dt'], alpha=SETTINGS['ridge_alpha']['B'])

R = lambda x, u: Rauton(np.atleast_2d(x)) + Br @ utils.phi(u, 1)

Frobenius norm of B_learn: 2527.0689


### Integrate model with inputs using learned influence of control (open-loop prediction)

Test open-loop prediction capabalities of SSM model

In [87]:
# Consolidate Test Data
zDataTest, yDataTest, uDataTest, tDataTest = [], [], [], []
for iTraj in indCntrlTest:
    tDataTest.append(y[iTraj][0])
    zDataTest.append(outData[iTraj][1])
    yDataTest.append(y[iTraj][1])
    uDataTest.append(inputData[iTraj][1])

test_data = (tDataTest, zDataTest, yDataTest, uDataTest)
traj_outDofs = [0, 1, 2]
test_results = utils.analyze_open_loop(test_data, model_save_dir, controlData, Wauton, R, Vauton, embed_coords=traj_outDofs, 
                        traj_coords=[0, 1, 2], PLOTS='save')

(like training data): RMSE = 4.3733
(overall): RMSE = 4.3733


Plot Gradients

In [89]:
dxDt_ROM_with_B = R(controlData['x_train'], controlData['u_train'])
utils.plot_gradients(controlData['t_train'], controlData['dxdt'], controlData['dxdt_ROM'], dxDt_ROM_with_B, PLOTS, model_save_dir)

## Save SSM model

In [91]:
utils.save_ssm_model(model_save_dir, RDInfo, IMInfo, Br, Vde, q_eq, u_eq, SETTINGS, test_results, custom_obs=True)

print("Saved SSM Model to: ", model_save_dir)

Saved SSM Model to:  /home/jalora/SSMR-for-control/ROM/python/hardware
